In [106]:
#INITIALIZA THE FUNCTIONS 
#%%
#import the necessary packages
import automatization
from automatization.ple_statistics import PleAuto
import pandas as pd
import os
import time
import json
from importlib import reload
import numpy as np
from automatization.ple_statistics import PleAuto
from matplotlib import pyplot as plt
reload(automatization.ple_statistics)
from automatization.ple_statistics import PleAuto
from scipy.constants import speed_of_light
import pickle
from pathlib import Path

import sys

sys.setrecursionlimit(10000)

pa = PleAuto(
    ple_gui, 
    ple_optimize_logic, 
    laser_scanner_logic,
    poi_manager_logic, 
    scanning_optimize_logic,
    spectrometer,  
    
    spectrometerlogic,
    cobolt,
    ibeam_smart
)

# def ple_is_here(res, center_sigma = 3e3, amplitude = 1000, sigma_stderr_ratio = 4, amplitude_stderr_ratio=3):
#     """Check if the ple is still there."""
#     it_is = True
#     if res["center"].stderr is None or res["sigma"].stderr is None or res["sigma"].value is None:
#         return False
#     if ( res["center"].stderr > center_sigma or 
#     res["sigma"].stderr * sigma_stderr_ratio > res["sigma"].value or 
#     res["amplitude"].stderr * amplitude_stderr_ratio > res["amplitude"].value or
#     res["amplitude"].value < amplitude): 
#         #ple is gone.
#         return False

#     return it_is
def ple_is_here(rsquared_bound = 0.5):
    """Check if the ple is still there."""
    
    if ple_gui.fit_result[1].rsquared < rsquared_bound:
        print("NO, PLE", ple_gui.fit_result[1].rsquared )
        return False
    
    else:
        return True


def adjust_eta(pa, poi_name, folder_defect, results_poi, center_v):
    """Adjust the etalon voltage to get the best fit."""
    # ple_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs["a"])
    ple_gui._mw.frequencyDoubleSpinBox.setValue(1000)
    
    eta_volts = [center_v, center_v + 0.2, center_v + 0.4, center_v + 0.6, center_v - 0.2, center_v - 0.4,  center_v - 0.6]
    r_squared = []
    for eta_v in eta_volts:
        laser_controller_remote.etalon_voltage = eta_v
        time.sleep(0.5)
        res = pa.do_ple_scan(lines = 1)
        time.sleep(0.5)
        r_squared.append(ple_gui.fit_result[1].rsquared)
    results_poi["eta_voltage"] = eta_volts[r_squared.index(max(r_squared))]
    laser_controller_remote.etalon_voltage = eta_volts[r_squared.index(max(r_squared))]
    res = pa.do_ple_scan(lines = 1)
    pa.save_ple(tag = "full_range_eta_adjusted",
                poi_name=poi_name, folder_name=folder_defect)

    ple_gui._mw.frequencyDoubleSpinBox.setValue(250)
    return res, results_poi
def ple_refocus(pa, opt_times = 1, 
                scan_frequency=200, 
                scan_resolution=500,
                scan_range = 5000):
    
    seqs = {str(seq): idx for idx, seq in enumerate(ple_gui._osd.settings_widget.available_opt_sequences)}
    ple_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs["a"])
    ple_gui._osd.change_settings({'scan_frequency': {"a": scan_frequency},
                                    "scan_resolution": {"a":scan_resolution},
                                    "scan_range": {"a": scan_range}}) #GHz

    ple_gui._osd.accept()
    time.sleep(0.8)
    pa.optimize_ple()
    return res
def settings_confocal_refocus(sequence = ["xy", "z"], scan_range_xy= 1e-6, scan_resolution = 25, scan_frequency = 25):
    seqs = {str(seq): idx for idx, seq in enumerate(scanner_gui._osd.settings_widget.available_opt_sequences)}
    scanner_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs["xy, z"])
    scanner_gui._osd.change_settings({'scan_frequency': {"x": scan_frequency, "y": scan_frequency, "z": scan_frequency},
                                    "scan_resolution": {"x": scan_resolution, "y": scan_resolution, "z":80},
                                    "scan_range": {"x": scan_range_xy, "y": scan_range_xy, "z": 4e-6}})

    scanner_gui._osd.accept()
    time.sleep(0.5)


def settings_confocal_refocus_coarse():
    seqs = {str(seq): idx for idx, seq in enumerate(scanner_gui._osd.settings_widget.available_opt_sequences)}
    scanner_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs["x, y, z"])

    scanner_gui._osd.change_settings({'scan_frequency': {"x": 5, "y": 5, "z": 5},
                                    "scan_resolution": {"x": 80, "y": 80, "z":80},
                                    "scan_range": {"x": 2.5e-6, "y": 2.5e-6, "z": 4.5e-6}})
    scanner_gui._osd.accept()
    time.sleep(0.5)

def confocal_refocus(opt_times=2):
    for i in range(opt_times):
        scanning_optimize_logic.start_optimize()
        while scanning_optimize_logic.module_state()=='locked':
            time.sleep(0.5)
    time.sleep(0.5)
#find the defect:
def find_the_defect(pa, poi_name, folder_defect):
    ple_gui._mw.frequencyDoubleSpinBox.setValue(1000)
    switchlogic.set_state("ScanningMode", 'Wavemeter')
    pa.set_resonant_power(power = 300)
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(2)
    time.sleep(1)
    settings_confocal_refocus_coarse()
    confocal_refocus(opt_times=2)

    
    switchlogic.set_state("ScanningMode", 'NI')
    time.sleep(0.5)
    #Check how the PLE look like
    res = pa.do_ple_scan(lines = 1)

    #configure slow scanning for the wavemeter scanning optimizations
    for kk in range(3):
        if not ple_is_here():
            switchlogic.set_state("ScanningMode", 'Wavemeter')
            time.sleep(0.5)
            settings_confocal_refocus_coarse()
            confocal_refocus(opt_times=1)
            #Check how the PLE look like
            time.sleep(0.5)
            switchlogic.set_state("ScanningMode", 'NI')
            res = pa.do_ple_scan(lines = 1)
            time.sleep(0.5)
            pa.save_ple(tag = "full_range_iter_{kk}",
                    poi_name=poi_name, folder_name=folder_defect)
            time.sleep(0.5)
        else:
            break

    pa.save_ple(tag = "full_range",
            poi_name=poi_name, 
            folder_name=folder_defect)
    time.sleep(0.2)
    ple_gui._mw.frequencyDoubleSpinBox.setValue(250)
    return res

def take_spectrum(pa, poi_name, folder_defect):
    # take spectrum to estimate SOC
    pa.set_resonant_power(power = 0)
    time.sleep(2)
    ibeam_smart.setPower(60000)
    ibeam_smart.enable()
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 100)
    pa.set_resonant_power(power = 0)
    time.sleep(1)
    pa.take_spectrum()

    
    
    time.sleep(1)
    spectrometer._mw.data_widget.fit_widget.sigDoFit.emit("DoubleLorentzian")
    time.sleep(1)
    pa.save_spectrum(name_tag=f"{poi_name}_blueNgreen", folder_path=folder_defect)
    # spectrometer.fit_results.params["center_1"].value
    if spectrometer.fit_results is not None:
        params = spectrometer.fit_results.params
        soc_ghz = float(speed_of_light / params["center_2"].value - speed_of_light / params["center_1"].value)
        center_1_ghz = float(params["center_1"].value)
        center_2_ghz = float(params["center_2"].value)
    ibeam_smart.disable()
    ibeam_smart.setPower(3000)
    #cobolt.disable_modulated()
    time.sleep(0.1)
    # results_poi["spectrum_fit_data"] = spectrometer.fit_results.best_fit
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 2)
    pa.set_resonant_power(power = 300)    
    time.sleep(1)
    return soc_ghz, center_1_ghz, center_2_ghz
# Perform the saturation measurement

def fine_optimize(pa, poi_name, folder_defect, results_poi):
    res = pa.do_ple_scan(lines = 1)
    pa.go_to_ple_target(res["center"].value)
    ple_refocus(pa, scan_range=4000, scan_frequency=500)
    settings_confocal_refocus_fine()
    confocal_refocus(opt_times=1)
    ple_refocus(pa, scan_range=4000, scan_frequency=500)
    confocal_refocus(opt_times=1)
    res = pa.do_ple_scan(lines = 1)
    pa.go_to_ple_target(res["center"].value)
    results_poi["center"] = res["center"].value
    
    results_poi["center_λ"] = high_finesse_wavemeter_remote.get_current_wavelength()
    pa.save_ple(tag = "full_range_optimized",
            poi_name=poi_name, 
            folder_name=folder_defect)
    return res, results_poi

def run_saturation_measurement(pa, res, poi_name, folder_defect, results_poi, large_sigma = 4000, small_sigma = 300, power_steps = None, low_power_steps = None, repumped_power = 100, no_repump_power = 150):
    ple_gui._mw.frequencyDoubleSpinBox.setValue(500)

    os.makedirs(saturation_folder := os.path.join(folder_defect, "saturation"), exist_ok=True)
    results_poi["saturation"] = {}
    idx_no_ple = None
    res_old = res
    if power_steps is None:
        power_steps = 3 * np.logspace(1.5, 2, 10, endpoint=True).astype(int)[::-1]
    if low_power_steps is None:
        low_power_steps = np.array([85, 78, 70, 65, 60])
    power_steps = np.append(power_steps,low_power_steps)
    zero_range = (
                res["center"].value - large_sigma,
                res["center"].value + large_sigma
            )
    res = pa.do_ple_scan(lines = 1, in_range=zero_range)
    for idx, power in enumerate(power_steps):
        os.makedirs(power_folder := os.path.join(saturation_folder, f"{power}"), exist_ok=True)
        
        if power > repumped_power: 
            cobolt.enable_modulated()
            pa.set_resonant_power(power = power)
            time.sleep(1)
            #align twice
            # for i in range(2)
            
            fine_range = (
                res["center"].value - small_sigma,
                res["center"].value + small_sigma
            )
            if res["center"].value is None or res["sigma"].value is None or not ple_is_here(rsquared_bound=0.2):
                res = pa.do_ple_scan(lines = 1, in_range=zero_range)
                
                if not ple_is_here(rsquared_bound=0.2):
                    continue
            res = pa.do_ple_scan(lines = 1, in_range=fine_range)
            # if abs(res_old["center"].value - res["center"].value) > res_old["sigma"].value*2:
            #     res = res_old
            # else:s
            #     res_old = res
            pa.save_ple(tag = f"{power}",
                poi_name=poi_name, folder_name=power_folder)

            results_poi.update({"saturation": 
                            {f"{power}_repump":
                                {"scan_data": ple_data_logic.last_saved_files_paths,
                                "sigma": res["sigma"].value,
                                "sigma_stderr": res["sigma"].stderr,
                                "center": res["center"].value
                                }
                                }})
        if (power < no_repump_power):
            os.makedirs(power_folder_norepump := os.path.join(power_folder, f"no_repump"), exist_ok=True)
            # check with initioalization
            cobolt.disable_modulated()
            pa.one_pulse_repump("violet")
            for i in range(3):
                pa.one_pulse_repump("violet")
                res_ = pa.do_ple_scan(lines = 1, in_range=fine_range)
                if not ple_is_here(rsquared_bound=0.4):
                    continue
            if not ple_is_here(rsquared_bound=0.4):
                    continue
            res_ = pa.do_ple_scan(lines = 5, in_range=fine_range)
            print("Saving no repump")
            pa.save_ple(tag = f"{power}_norepump",
                poi_name=poi_name, folder_name=power_folder_norepump)
            results_poi.update({"saturation": 
                            {f"{power}_norepump":
                                {"scan_data": ple_data_logic.last_saved_files_paths,
                                "sigma": res_["sigma"].value,
                                "sigma_stderr": res_["sigma"].stderr,
                                "center": res_["center"].value
                                }
                                }})
        if power <= 40:
            breaks

            #save_plots
    return results_poi

def check_factor_sigma():
    # constraints
    scan_range_constr = laser_scanner_logic.scan_ranges["a"] 
    pa.go_to_ple_target(scan_range_constr[0])
    time.sleep(5)
    w1 = high_finesse_wavemeter_remote.get_current_wavelength()
    
    pa.go_to_ple_target(scan_range_constr[1])
    time.sleep(5)
    w2 = high_finesse_wavemeter_remote.get_current_wavelength()

    factor_sigma = (w2 - w1) * 1000000 / (scan_range_constr[1] - scan_range_constr[0]) #GHz 
    return factor_sigma

# constraints
scan_range_constr = laser_scanner_logic.scanner_constraints.axes["a"].value_range
scanning_optimize_logic._backwards_line_resolution = 20


In [2]:
folder = rf"C:\Users\yy3\Documents\data\Vlad\158\03-05\#1A\Auto"
os.makedirs(folder, exist_ok=True)
#folder = os.path.join(folder, r"auto2")
coarse_sigma = 4000
center_v = laser_controller_remote.etalon_voltage
os.makedirs(folder, exist_ok=True)
high_finesse_wavemeter_remote.start_acquisition()
# run throught the defects:
cobolt.enable_modulated()
for poi_name in poi_manager_logic.poi_names:
    if poi_name == "ref":
        continue
    results_poi = {}
    results_poi[poi_name] = {}  
    
    os.makedirs(folder_defect := os.path.join(folder, poi_name),  exist_ok=True)
    #Go to the defect:
    cobolt.set_laser_modulated_power(5)
    settings_confocal_refocus_fine()
    pa.go_to_poi(poi_name, opt_times=0, ref_poi=None) 
    #
    print("Find the defect ", poi_name)
    res = find_the_defect(pa, poi_name, folder_defect)

    print("Run spectrum")
    results_poi = take_spectrum(pa, poi_name, folder_defect, results_poi)

    print("Run the saturation measurements")
    cobolt.set_laser_modulated_power(1)
    
    ple_gui._mw.frequencyDoubleSpinBox.setValue(500)
    ple_gui._mw.resolutionDoubleSpinBox.setValue(2000)
    pa.go_to_ple_target(laser_scanner_logic.scan_ranges["a"][0])

    power = 300
    pa.set_resonant_power(power=power)
    cobolt.enable_modulated()
    # pa.one_pulse_repump("violet")
    res = pa.do_ple_scan(lines=1)
    pa.go_to_ple_target(res["center"].value)
    time.sleep(1)
    w1 = high_finesse_wavemeter_remote.get_current_wavelength()
    c = 299792458

    print(1e-3 * c / w1, "nm")
    np.savetxt(os.path.join(folder_defect, "lambda.txt"), [1e-3 * c / w1])
    pa.save_ple(tag=f"{power}_full_range", poi_name=poi_name, folder_name=folder_defect)

    pa.set_resonant_power(power=120)
    for i in range(3):
        if not ple_is_here(rsquared_bound=0.5):
            continue
        pa.one_pulse_repump("violet")
        res = pa.do_ple_scan(lines=1, in_range=(res["center"].value - coarse_sigma, res["center"].value + coarse_sigma))

    if not ple_is_here(rsquared_bound=0.5):
        continue
    pa.save_ple(tag=f"{power}_coarse_range", poi_name=poi_name, folder_name=folder_defect)

    

    
    # with open(os.path.join(folder_defect, f'results_{poi_name}'), 'wb') as handle:
    #     pickle.dump(results_poi, handle, protocol=pickle.HIGHEST_PROTOCOL)

Find the defect  def1
NO, PLE 0.06964438361909386
NO, PLE 0.021489602501675087
NO, PLE 0.013479882282027567
Run spectrum
Run the saturation measurements
619.2317355709993 nm
NO, PLE 0.01133878461440807
NO, PLE 0.01133878461440807
NO, PLE 0.01133878461440807
Find the defect  def2
NO, PLE 0.10816066429224203
NO, PLE 0.006914118024270111
NO, PLE 0.013280369802660119
Run spectrum
Run the saturation measurements
619.2254946855176 nm
NO, PLE 0.013596203636346393
NO, PLE 0.013596203636346393
NO, PLE 0.013596203636346393
NO, PLE 0.013596203636346393
Find the defect  def3
NO, PLE 0.3235541829884395
NO, PLE 0.0003369472070007262
NO, PLE 0.00048250220676737676
Run spectrum
Run the saturation measurements
619.2303379926016 nm
NO, PLE 0.018297821981533402
NO, PLE 0.018297821981533402
NO, PLE 0.018297821981533402
NO, PLE 0.018297821981533402
Find the defect  def4
Run spectrum
Run the saturation measurements
619.2266199829509 nm
NO, PLE 0.07983616177732422
NO, PLE 0.07983616177732422
NO, PLE 0.079836

KeyboardInterrupt: 

In [3]:
ibeam_smart.enable()

In [8]:
ibeam_smart.disable()

In [7]:
ibeam_smart.setPower(30000)

In [12]:
cobolt.set_laser_modulated_power(5)
cobolt.enable_modulated()

# 1 Full power full range

In [109]:
folder = r"C:\Users\yy3\Documents\data\Vlad\158\03-05\#1A"

In [110]:
poi_name = "def5"


os.makedirs(folder_defect := os.path.join(folder, poi_name),  exist_ok=True)

ple_gui._save_folderpath = folder_defect


ple_gui.save_path_widget.currPathLabel.setText(ple_gui._save_folderpath)
ple_gui.save_path_widget.newPathCheckBox.setChecked(False)

In [111]:
switchlogic.set_state("ScanningMode", 'Wavemeter')
cobolt.set_laser_modulated_power(2)
cobolt.enable_modulated()
pa.set_resonant_power(300)


In [112]:
#Find the defect
settings_confocal_refocus(scan_range_xy=3e-6)
confocal_refocus(opt_times=2)

In [96]:
#Optimize the defect
switchlogic.set_state("ScanningMode", 'NI')
res = pa.do_ple_scan(lines = 1)
pa.go_to_ple_target(res["center"].value)
ple_refocus(pa, scan_range=4000, scan_frequency=500)
settings_confocal_refocus(scan_range_xy=1e-6, scan_frequency=50)
confocal_refocus(opt_times=1)
ple_refocus(pa, scan_range=4000, scan_frequency=500)
confocal_refocus(opt_times=1)



res = pa.do_ple_scan(lines = 1)
pa.go_to_ple_target(res["center"].value)
pa.save_ple(tag = f"full_range_power_{power}", 
            poi_name=poi_name, folder_name=folder_defect)

time.sleep(1)
w1 = high_finesse_wavemeter_remote.get_current_wavelength()
buf = high_finesse_wavemeter_remote._wavelength_buffer
print(1e-3 * c / w1, "nm")

line = f"lambda = {1e-3 * c / w1} nm\n"

with open(os.path.join(folder_defect, "info.txt"), "a") as f:
    f.write(line)
np.savetxt(os.path.join(folder_defect, "wavelength_buffer.txt"), high_finesse_wavemeter_remote._wavelength_buffer)



soc_ghz, center_1_ghz, center_2_ghz = take_spectrum(pa, poi_name, folder_defect)

line = f"\nsoc_ghz = {soc_ghz} GHz, \ncenter_1_ghz = {center_1_ghz} GHz, \ncenter_2_ghz = {center_2_ghz} GHz\n"
with open(os.path.join(folder_defect, "info.txt"), "a") as f:
    f.write(line)

factor_sigma = check_factor_sigma()
line = f"\nfactor_sigma = {factor_sigma}\n"
with open(os.path.join(folder_defect, "info.txt"), "a") as f:
    f.write(line)


for power in [300, 250, 200, 150, 120]:
    pa.set_resonant_power(power)
    res = pa.do_ple_scan(lines = 1, in_range=(
        res["center"].value - SCAN_RANGE, res["center"].value + SCAN_RANGE))
    
    pa.save_ple(tag = f"power_{power}", 
                poi_name=poi_name, folder_name=folder_defect)
    if not ple_is_here():
        print("Last power", power)
        last_power = power
        break


cobolt.disable_modulated()
for power in np.linspace(last_power, 100, 3, endpoint=True):
    pa.go_to_ple_target(res["center"].value)
    pa.set_resonant_power(power)
    pa.one_pulse_repump("violet")
    res = pa.do_ple_scan(lines = 1, in_range=(
        res["center"].value - SCAN_RANGE, res["center"].value + SCAN_RANGE))
    pa.save_ple(tag = f"power_{power}_norepump", 
                poi_name=poi_name, folder_name=folder_defect)
    if not ple_is_here(rsquared_bound=0.2):
        print("Last power", power)
        break
